#### Hinweise zum Jupyter Notebook:
- SHIFT-ENTER: Ausführung der aktuellen Zelle und Wechsel zur nächsten Zelle
- Reihenfolge der Zellen beachten [xxx] 
- Zur Dokumentation __Markdown__ wählen
    - __ vor und nach Wort --> Fett
    - _ vor und nach Wort --> kursiv
    - "# oder ##, ###", usw. steht für eine Überschrift
> Help --> Markdown References
- Speichere das Notebook, indem du es herunterlädst und speicherst 
    > File --> Download


---

# Grundlagen der Datenbankerstellung (mit SQL(ite))

## Unterschied SQL und SQLite
- SQL ist ein serverseitiges DBMS
- SQLite läuft serverlos direkt in einer Anwendung (z.B. in einem Jupyter Notebook) und Änderungen werden im Browser-Cache gespeichert.
- SQLite --> Einsatz in eingebetteten Datenbanksystemen, daher fehlen Funktionen wie die Möglichkeit, Objektberechtigungen zu verwalten (GRANT, REVOKE)
- __WICHTIG:__ Es geht stets nur ein Befehl pro Zeile und nur eine Datenbank pro Kernel/Notebook!

### Erstellung einer "In-Memory" - Datenbank und Aktivierung der Fremdschlüsselnutzung

In [14]:
%CREATE sportDB.db

In [15]:
PRAGMA foreign_keys = ON;

---

## Erstellung einer neuen Tabelle
- CREATE TABLE __name__(__Attribut__ TYP)
- __PRIMARY KEY__ zeigt den Primärschlüssel an. Dieser muss dadurch immer verschieden sein und nicht null.
- Attribute, die nicht leer sein dürfen, werden mit __NOT NULL__ gekennzeichnet.
- __FOREIGN KEY(TeamID) REFERENCES team(ID)__: TeamID ist ein Fremdschlüssel aus Tabelle team mit Attribut ID. Dadurch kann __kein__ Spieler erstellt werden, bei dem es das Team nicht gibt. Auch kann kein Team gelöscht werden, wenn es noch Spieler darin gibt.
#### Datentypen:
- NULL (keine Angabe, falls unbekannt)
- INTEGER (Zahlen)
- REAL (Dezimalzahlen)
- CHAR(40) (Text mit Länge der Zeichen (40))

In [16]:
CREATE TABLE team (
    ID INTEGER PRIMARY KEY,
    Name CHAR(40), 
    Platz INTEGER, 
    Farbe CHAR(40))

In [17]:
CREATE TABLE spieler (
    Name CHAR(40) PRIMARY KEY, 
    TeamID INTEGER NOT NULL, 
    Level INTEGER NOT NULL, 
    Treffer INTEGER,
    FOREIGN KEY(TeamID) REFERENCES team(ID)
)

---

### Einfügen eines neuen Datensatzes

- __INSERT INTO__ tabellenName (Attribute optional) __VALUES__ (Wert1, Wert2, ...);
- Strings in "..."

> Tipp: Zellen duplizieren!


In [22]:
INSERT INTO spieler (Name, TeamID, Level, Treffer) VALUES ("Bob", 1, 3, 23);

In [23]:
INSERT INTO spieler (Name, TeamID, Level, Treffer) VALUES ("Alice", 1, 5, 33);

In [24]:
INSERT INTO spieler (Name, TeamID, Level, Treffer) VALUES ("Rob", 1, 7, 12);

In [25]:
INSERT INTO spieler (Name, TeamID, Level, Treffer) VALUES ("Anna", 1, 5, 37);

In [26]:
INSERT INTO spieler (Name, TeamID, Level, Treffer) VALUES ("Tim", 2, 2, 11);

In [27]:
INSERT INTO spieler (Name, TeamID, Level, Treffer) VALUES ("Billy", 2, 1, 28);

In [28]:
INSERT INTO spieler (Name, TeamID, Level, Treffer) VALUES ("Sabine", 2, 7, 27);

In [29]:
INSERT INTO spieler (Name, TeamID, Level, Treffer) VALUES ("Lissel", 2, 10, 29);

In [30]:
-- Möchte man jeden Wert eintragen, kann die Attribut-Klammer weggelassen werden!
INSERT INTO team VALUES (1, "BVB", 3, "Gelb");

Error: UNIQUE constraint failed: team.ID

In [31]:
INSERT INTO team VALUES (2, "Dynamo Desden", 1, "Rot");

Error: UNIQUE constraint failed: team.ID

In [34]:
SELECT * FROM spieler

Name,TeamID,Level,Treffer
Bob,1,3,23
Alice,1,5,33
Rob,1,7,12
Anna,1,6,37
Tim,2,2,11
Billy,2,1,28
Sabine,2,7,27
Lissel,2,10,29


In [33]:
-- Update der Tabelle Spieler, Setze Level um 1 hoch WO Name = Anna
Update Spieler Set Level = Level + 1 where Name = "Anna"

In [36]:
SELECT 
    team.Name, 
    COUNT(*) as AnzahlSpieler, 
    SUM(spieler.Treffer) as AnzahlTreffer,
    AnzahlTreffer/AnzahlSpieler as Schnitt
FROM 
    spieler, 
    team 
where 
    spieler.TeamID = team.ID 
Group by team.Name
ORDER BY AnzahlTreffer ASC; 
-- DESC/ASC = Absteigend/Aufsteigend

Error: no such column: AnzahlTreffer

---

### Beispielabfragen
__Tipp:__
Erstellt euch komplizierte Abfragen mit ChatGPT! Übergebt dafür die Tabellenerstellungs-Abfrage in die Prompt!

In [ ]:
--- Durchschnittlicher Treffer pro Spieler pro Team:
SELECT t.Name AS TeamName, AVG(s.Treffer) AS AverageHitsPerPlayer
FROM team t
JOIN spieler s ON t.ID = s.TeamID
GROUP BY t.Name;

In [ ]:
--- Anzahl der Spieler pro Team mit einem bestimmten Level:
SELECT t.Name AS TeamName, COUNT(*) AS NumberOfPlayers, s.Level
FROM team t
JOIN spieler s ON t.ID = s.TeamID
GROUP BY t.Name, s.Level;

In [ ]:
---Teams mit mindestens 3 Spielern und deren Platzierung:
SELECT team.Name AS TeamName, team.Platz
FROM team
WHERE team.ID IN (SELECT TeamID FROM spieler GROUP BY TeamID HAVING COUNT(*) >= 3);

In [ ]:
---Durchschnittliche Treffer pro Level:
SELECT Level, AVG(Treffer) AS AverageHitsPerLevel
FROM spieler
GROUP BY Level;

---
---

### Einfache grafische Darstellung mit __XVega__

%XVEGA_PLOT 

    X_FIELD LvL      --> x-Achsen Attribut
    Y_FIELD Points   --> y-Achsen Attribut
    MARK circle      --> Datenpunkte Style (ARC,AREA,BAR,CIRCLE,LINE,POINT,RECT,RULE,SQUARE,TICK,TRAIL)
    COLOR green      --> Farbe der Punkte/Markierung
    WIDTH 500        --> Breite
    HEIGHT 200       --> Höhe
    <> 
    SELECT Level as LvL, Treffer as Points FROM spieler  --> SQL-Abfrage mit den oben genannten Attributen/ __as__ benennt um (praktisch für Aggregatsfunktionen)
    
__WICHTIG__: Attribute nach _Select_ müssen nummerisch sein!

In [ ]:
%XVEGA_PLOT
    X_FIELD Name
    Y_FIELD AnzahlTreffer
    MARK bar
    COLOR green
    WIDTH 100
    HEIGHT 200
    <>
  SELECT team.ID as Name, SUM(spieler.Treffer) as AnzahlTreffer FROM spieler, team where spieler.TeamID = team.ID Group by team.Name

---
---

### __Arbeitsauftrag__
1. Erstelle ein neues Notebook!
2. Übertrage dein Datenmodell aus dem Relationenmodell in eine SQLite Datenbank.
3. Verknüpfe die Tabellen über die Fremdschlüssel-Klausel.
4. Füge je Tabelle 3-4 Datensätze hinzu.
5. Erstelle eine Abfrage mit den Agregatsfunktionen SUM/AVG/COUNT und GROUB BY/HAVING
6. Lass eine Abfrage grafisch darstellen!
7. Lade das Notebook herunter und lege es in den Abgabeordner!